**Changing the Directory to saved Model Directory**

In [1]:
cd /content/drive/MyDrive/Mini Project

/content/drive/MyDrive/Mini Project


**Change Permission to run ngrok**

In [2]:
!chmod +x ngrok

**Adding authtoken of ngrok**

In [3]:
!./ngrok authtoken <your ngrok authtoken>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


**Start the ngrok server**

In [4]:
import subprocess
import time
import requests

# Start ngrok
ngrok_process = subprocess.Popen(['./ngrok', 'http', '5000'])

# Sleep for a few seconds to allow ngrok to start
time.sleep(5)

try:
    # Get ngrok URL using ngrok API
    ngrok_url = 'http://localhost:4040/api/tunnels'
    response = requests.get(ngrok_url)
    ngrok_data = response.json()

    # Extract the ngrok URL
    ngrok_url = ngrok_data['tunnels'][0]['public_url']
    print("Ngrok URL:", ngrok_url)
except Exception as e:
    print("Failed to retrieve ngrok URL:", e)


Ngrok URL: https://2e0e-34-69-162-232.ngrok-free.app


**Importing Necessary Libraries**

In [5]:
# from flask import Flask, request, jsonify, render_template
# # from flask_ngrok import run_with_ngrok
# from tensorflow.keras.models import load_model
# from PIL import Image
# import numpy as np
# import io
# import base64

**Creating Flask Application**

In [ ]:
from flask import Flask, render_template, request
from tensorflow.keras.models import load_model
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib.pyplot as plt
from PIL import Image
import base64
import io
import numpy as np
import pickle

app = Flask(__name__)

# Load the model
loaded_model = load_model('Fake_image_detection_model.h5')  # Loading model

# Load training history from a saved file
with open('training_history.pkl', 'rb') as file:
    history = pickle.load(file)

# Function to generate accuracy and loss graph
def generate_graph():
    epochs = range(len(history['accuracy']))
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].plot(epochs, history['accuracy'], label='Training Accuracy')
    ax[0].plot(epochs, history['val_accuracy'], label='Validation Accuracy')
    ax[0].set_title('Training and Validation Accuracy')
    ax[0].legend()
    ax[1].plot(epochs, history['loss'], label='Training Loss')
    ax[1].plot(epochs, history['val_loss'], label='Validation Loss')
    ax[1].set_title('Training and Validation Loss')
    ax[1].legend()

    # Save the plot to a BytesIO object
    img = io.BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plt.close()

    return base64.b64encode(img.getvalue()).decode('utf-8')

@app.route('/', methods=['GET', 'POST'])
def home():
   return render_template('index_new.html')

@app.route('/prediction.html', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        img = Image.open(file.stream)
        img = img.resize((150, 150))
        img = np.array(img) / 255.0
        img = np.reshape(img, (1, 150, 150, 3))
        prediction = loaded_model.predict(img)
        if prediction[0][0] > 0.5:
            result = "Real Image"
            confidence_percentage = f"{prediction[0][0] * 100:.2f}%"
            reason = "This image appears to contain natural and consistent patterns, consistent with real-world scenes."
        else:
            result = "Fake Image"
            confidence_percentage = f"{100-prediction[0][0] * 100:.2f}%"
            reason = "This image exhibits characteristics commonly found in images generated by StyleGAN, including surreal and non-realistic features often associated with artificially generated content."

        # Convert image to base64 to display in HTML
        img = Image.fromarray((img.reshape(150, 150, 3) * 255).astype(np.uint8))
        image = io.BytesIO()
        img.save(image, format='PNG')
        image.seek(0)
        encoded_img = base64.b64encode(image.getvalue()).decode('utf-8')

        return render_template('index.html', prediction=result, confidence=confidence_percentage, image=encoded_img, reason=reason)
    return render_template('index.html', prediction='')

@app.route('/model_summary')
def model_summary():
    summary = []
    loaded_model.summary(print_fn=lambda x: summary.append(x))
    return render_template('model_summary.html', summary=summary)

@app.route('/model_performance')
def model_performance():
    graph = generate_graph()
    return render_template('model_performance.html', graph=graph)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 09:57:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 09:57:39] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 09:57:42] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 09:57:42] "GET /prediction.html HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 09:57:43] "GET /static/predict.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 10:44:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 10:44:22] "GET /static/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 10:44:23] "GET /prediction.html HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 10:44:24] "GET /static/predict.css HTTP/1.1" 304 -


1/1 [==============================] - 1s 715ms/step


INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 11:08:31] "POST /prediction.html HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 11:08:32] "GET /static/predict.css HTTP/1.1" 304 -


1/1 [==============================] - 0s 69ms/step


INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 11:09:00] "POST /prediction.html HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 11:09:01] "GET /static/predict.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 11:11:23] "GET /model_summary HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 11:11:23] "GET /static/predict.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 11:11:24] "GET /prediction.html HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 11:11:25] "GET /static/predict.css HTTP/1.1" 304 -
